In [4]:
import pandas as pd
all_stock_list=['BA','FDX','AMP','COP','AAPL','TSLA','OXY','XOM','JASIF','SCC','BDMS','FORTH','HANA','BTS','CPALL','MINT']
for i,value in enumerate(all_stock_list):
    locals()[all_stock_list[i]+'_train'] = pd.read_csv(all_stock_list[i]+'_real_training_signal.csv')

In [8]:
dfo=AAPL_train
dfo.drop(dfo.columns[[0]],axis=1,inplace=True)
dfo

,ma30_50_signal,ma30_200_signal,ma50_200_signal,obv_signal,macd_signal,adx_signal,stoch_signal,roc_signal,rsi_signal,cci_signal,atr_signal,wpr_signal,bb_signal,cdc_signal,sar_signal,close_price,adj_close_price
0,1,1,1,1,0,0,0,-1,0,0,-1,0,0,-0.5,1,43.064999,40.950497
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,-1,43.057499,40.943356
2,0,0,0,0,0,0,-1,0,0,0,0,0,0,1.0,0,43.257500,41.133549
3,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0,0,43.750000,41.601860
4,0,0,0,0,0,0,0,-1,0,0,0,0,0,1.0,0,43.587502,41.447346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,0,0,0,0,0,0,-1,0,0,0,0,0,0,1.0,0,180.330002,179.289459
1004,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,179.289993,178.255417
1005,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,179.380005,178.344925
1006,0,0,0,0,0,0,0,-1,0,0,0,0,0,1.0,0,178.199997,177.171738


In [9]:
sl1=dfo['ma30_50_signal']
sl2=dfo['ma30_200_signal']
sl3=dfo['ma50_200_signal']
sl4=dfo['obv_signal']
sl5=dfo['macd_signal']
sl6=dfo['adx_signal']
sl7=dfo['stoch_signal']
sl8=dfo['roc_signal']
sl9=dfo['rsi_signal']
sl10=dfo['cci_signal']
sl11=dfo['atr_signal']
sl12=dfo['wpr_signal']
sl13=dfo['bb_signal']
sl14=dfo['cdc_signal']
sl15=dfo['sar_signal']

In [10]:
from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,td = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10+w11+w12+w13+w14+w15
nlc = NonlinearConstraint(const, 1 ,1)

In [17]:
import sympy as sp

In [18]:
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')
w10=sp.symbols('w11')
w10=sp.symbols('w12')
w10=sp.symbols('w13')
w10=sp.symbols('w14')
w10=sp.symbols('w15')
td=sp.symbols('td')

In [21]:
import numpy as np

def de(fobj, bounds, mut=0.5, crossp=0.8, popsize=80, its=3000):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
        yield best, fitness[best_idx]

In [ ]:
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint, Bounds


bounds = [[0, 1], [0, 1],[0, 1], [0, 1],[0, 1], [0, 1],[0, 1], [0, 1],[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1],[0,0.5]]
def func(p):
    global S,B
    S=0
    B=0
    ret_l=[]
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,td = p
    mon=100000
    amount=0
    for i in range(0,len(dfo)):
        r = ((w1*sl1[i]+w2*sl2[i]+w3*sl3[i]+w4*sl4[i]+w5*sl5[i]+w6*sl6[i]+w7*sl7[i]+w8*sl8[i]+w9*sl9[i]+w10*sl10[i]+w11*sl11[i]+w12*sl12[i]+w13*sl13[i]+w14*sl14[i]+w15*sl15[i])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10+w11+w12+w13+w14+w15))
        if i!=0:
            if dfo['adj_close_price'][i]<dfo['adj_close_price'][i-1]*0.2:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
        if r>td:
            sumall=mon//dfo['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*dfo['adj_close_price'][i]*1.002
            B+=1
        elif r<-td:
            mon+=amount*dfo['adj_close_price'][i]*0.998
            amount=0
            S+=1
        else:
            pass
        ret_l.append(mon+amount*dfo['adj_close_price'][i])
    ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
    return -ret

In [ ]:
def fobj():
    result = differential_evolution(func, bounds,constraints=(nlc),maxiter=3000,popsize=100,recombination=0.8)
    print(result)

In [ ]:
fobj()

In [ ]:
result = list(de(func, bounds=[(0,1)]*15+[(0,0.5)],its=3000))
print(result[-1])